In [8]:
'''
An imputation method based on ranking and nearest neighbor.
    
Beretta L, Santaniello A. Nearest neighbor imputation algorithms: a critical
evaluation. BMC Med Inform Decis Mak. 2016 Jul 25;16 Suppl 3:74.
'''
#Author: Lorenzo Beretta, lorberimm@hotmail.com

#Adapted by L Kahn; 11/20/18

#Dependencies Installation
#pip install rkNN-imputer, pip install sk-rebate

import sys
import numpy as np
import pandas as pd
from sklearn import neighbors


class Imputer:
    """Imputer class."""

    def _fit(self, X, column, k=10, is_categorical=False):
        """Fit a knn classifier for missing column.
        - Args:
                X(numpy.ndarray): input data
                column(int): column id to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                clf: trained k nearest neighbour classifier
        """
        clf = None
        if not is_categorical:
            clf = neighbors.KNeighborsRegressor(n_neighbors=k)
        else:
            clf = neighbors.KNeighborsClassifier(n_neighbors=k)
        # use column not null to train the kNN classifier
        missing_idxes = np.where(pd.isnull(X[:, column]))[0]
        if len(missing_idxes) == 0:
            return None
        X_copy = np.delete(X, missing_idxes, 0)
        X_train = np.delete(X_copy, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_train[:, col_id]))[0]
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_train[col_missing_idxes, col_id] = col_mean[col_id]
        y_train = X_copy[:, column]
        # fit classifier
        clf.fit(X_train, y_train)
        return clf

    def _transform(self, X, column, clf, is_categorical):
        """Impute missing values.
        - Args:
                X(numpy.ndarray): input numpy ndarray
                column(int): index of column to be imputed
                clf: pretrained classifier
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X(pandas.dataframe): imputed dataframe
        """
        missing_idxes = np.where(np.isnan(X[:, column]))[0]
        X_test = X[missing_idxes, :]
        X_test = np.delete(X_test, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        # fill missing values in each column with current col_mean
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_test[:, col_id]))[0]
            # if no missing values for current column
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_test[col_missing_idxes, col_id] = col_mean[col_id]
        # predict missing values
        y_test = clf.predict(X_test)
        X[missing_idxes, column] = y_test
        return X

    def knn(self, X, column, k=10, is_categorical=False):
        """Impute missing value with knn.
        - Args:
                X(pandas.dataframe): dataframe
                column(str): column name to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X_imputed(pandas.dataframe): imputed pandas dataframe
        """
        X, column = self._check_X_y(X, column)
        clf = self._fit(X, column, k, is_categorical)
        if clf is None:
            return X
        else:
            X_imputed = self._transform(X, column, clf, is_categorical)
            return X_imputed

    def _check_X_y(self, X, column):
        """Check input, if pandas.dataframe, transform to numpy array.
        - Args:
                X(ndarray/pandas.dataframe): input instances
                column(str/int): column index or column name
        - Returns:
                X(ndarray): input instances
        """
        column_idx = None
        if isinstance(X, pd.core.frame.DataFrame):
            if isinstance(column, str):
                # get index of current column
                column_idx = X.columns.get_loc(column)
            else:
                column_idx = column
            X = X.as_matrix()
        else:
            column_idx = column
        return X, column_idx

In [6]:
import numpy as np
from rkNN_imputer.rkNN import rkNN

dummy = np.array ((["0.2", "0.4", "5", "0.4", "6"],
                ["?", "0.4", "8", "0.5", "8"],
                ["0.1", "0.8", "3", "0.2", "6"],
                ["0.3", "0.2", "?", "0.1", "7"],
                ["0.2", "0.9", "3", "0.2", "4"]))

imp = rkNN ()
imp.fit (dummy)
imputed_dummy = imp.transform (dummy)

# show orignal and imputed dataset
print (dummy)
print ("")
print (imputed_dummy)

None
None
[['0.2' '0.4' '5' '0.4' '6']
 ['?' '0.4' '8' '0.5' '8']
 ['0.1' '0.8' '3' '0.2' '6']
 ['0.3' '0.2' '?' '0.1' '7']
 ['0.2' '0.9' '3' '0.2' '4']]

[[0.2 0.4 5.  0.4 6. ]
 [0.1 0.4 8.  0.5 8. ]
 [0.1 0.8 3.  0.2 6. ]
 [0.3 0.2 4.  0.1 7. ]
 [0.2 0.9 3.  0.2 4. ]]


In [17]:
#Import Coffee Rust Data

#1584 weekly observations from Brasil from January 1, 1991-July 30, 2018

#Data Acquisition described at: https://github.com/thedatalass.
brasildf = pd.read_csv("Coffee-Rust.csv")
brasildf.head()

,Date,Temp(C),Precipitation(mm),Rust (%),Production,Futures
0,1/1/1991,25.643,260.878,15.00,2273.83,0.8805
1,1/8/1991,25.643,260.878,15.00,2273.83,0.9045
2,1/15/1991,25.643,260.878,15.00,2273.83,0.8610
3,1/22/1991,25.643,260.878,15.00,2273.83,0.8670
4,1/29/1991,25.643,260.878,15.00,2273.83,0.8315


In [19]:
#Convert pandas dataframe as numpy array
brasildf.values

array([['1/1/1991', 25.643, 260.878, '15.00', 2273.83, 0.8805],
       ['1/8/1991', 25.643, 260.878, '15.00', 2273.83, 0.9045],
       ['1/15/1991', 25.643, 260.878, '15.00', 2273.83, 0.861],
       ...,
       ['7/16/2018', 24.7691, 6.0, '?', 5000.0, 1.103],
       ['7/23/2018', 24.7691, 6.0, '?', 5000.0, 1.1165],
       ['7/30/2018', 24.7691, 6.0, '?', 5000.0, 1.114]], dtype=object)